# Tightness-Adjusted Flows


In [ ]:
import matplotlib.pyplot as plt
from pathlib import Path
from matplotlib.collections import LineCollection
import matplotlib as mpl
import numpy as np
from scipy import optimize as opt
from scipy import integrate as intg
from scipy.optimize import least_squares
from scipy.stats import beta
from scipy.stats import cumfreq, beta

%matplotlib inline
plt.style.use('../notebook.mplstyle')

## Exogenous Parameters


In [ ]:
# Setting exogenous parameters
def reset_exog_params():
    global Bm, Bw, bm_vals, bw_vals, δ, Um, Uw, Fm, Fw, λm, λw  
    Bm = 10
    Bw = 10
    bm_vals = range(1,Bm+1) 
    bw_vals = range(1,Bw+1)
    
    δ = 0.97
    Um = lambda θ : θ 
    Uw = lambda θ : θ
    Fm = beta(3,3)
    Fw = beta(3,3)
    λm = 10
    λw = 10

## Two-Sided Search Equilibrium Conditions 


In [ ]:
# Optimality conditions
def SSE(x): 
    # Endogenous Variables 
    μ = x[:Bm] 
    ω = x[Bm:Bm+Bw]  
    print('mu:', μ)
    print('omega:', ω)

    Nm = x[Bm+Bw:2*Bm+Bw]
    Nw = x[2*Bm+Bw:] 
    
    print('Nm:', Nm)
    print('Nw:', Nw)
    
    τm = min(Nw.sum()/Nm.sum(), 1)
    τw = τm * (Nm.sum()/Nw.sum()) 
    αm = (τm*δ)/(1-δ*(1-τm))
    αw = (τw*δ)/(1-δ*(1-τw)) 

    # Initialising system of equilibrium equations
    E = np.empty(2*Bm + 2*Bw + 2) 

    # Initial condition for men 
    E[0] = (Um(μ[0]) 
            - αm * Um(μ[0]) * Fw.cdf(μ[0]) 
            - αm * intg.quad(lambda t: Um(t) * Fw.pdf(t), μ[0], 1)[0]) 
    
    # Intertemporal optimality conditions for men
    for b in range(1, Bm):
        E[b] = (Um(μ[b]) 
                - αm * Um(μ[b]) * Fw.cdf(μ[b]) 
                - αm * Um(μ[b-1])*(1-Fw.cdf(μ[b-1])) 
                - αm * intg.quad(lambda t : Um(t) * Fw.pdf(t), μ[b], μ[b-1])[0])

    E[Bm] = (Uw(ω[0]) 
            - αw * Uw(ω[0]) * Fm.cdf(ω[0]) 
            - αw * intg.quad(lambda t: Uw(t) * Fm.pdf(t), ω[0], 1)[0])

    # Intertemporal optimality conditions for women
    for b in range(1, Bw):
        E[Bm+b] = (Uw(ω[b]) 
                - αw * Uw(ω[b]) * Fm.cdf(ω[b]) 
                - αw * Uw(ω[b-1])*(1-Fm.cdf(ω[b-1])) 
                - αw * intg.quad(lambda t : Uw(t) * Fm.pdf(t), ω[b], ω[b-1])[0]) 
                
    # Platform flow men
    E[Bm+Bw] = λm - (1-δ)*Nm.sum() - Nm[0]*δ*τm*(1-Fw.cdf(μ[0]))

    for b in range(0, Bm-1):
       E[Bm+Bw+b+1] = (Nm[b+1]*δ*τm*(1-Fw.cdf(μ[b+1]))
                    - Nm[b]*(1-δ) - Nm[b]*δ*τm*(1-Fw.cdf(μ[b])))

    E[2*Bm+Bw] = λm - Nm[Bm-1]*(1-δ) - Nm[Bm-1]*δ*τm*(1-Fw.cdf(μ[Bm-1]))

    # Platform flow women
    E[2*Bm+Bw+1] = λw - (1-δ)*Nw.sum() - Nw[0]*δ*τw*(1-Fm.cdf(ω[0]))  

    for b in range(0, Bw-1):
       E[2*Bm+Bw+2+b] = (Nw[b+1]*δ*τw*(1-Fm.cdf(ω[b+1]))
                      - Nw[b]*(1-δ) - Nw[b]*δ*τw*(1-Fm.cdf(ω[b])))

    E[2*Bm+2*Bw+1] = λw - Nw[Bw-1]*(1-δ) - Nw[Bw-1]*δ*τw*(1-Fm.cdf(ω[Bw-1])) 

    return E 

## Solving For Steady State Equilibria 

In [ ]:
reset_exog_params()
m_test = np.random.rand(Bm) 
w_test = np.random.rand(Bw) 

nm_test = np.random.randint(0, λm, size=Bm)
nw_test = np.random.randint(0, λw, size=Bw)

print('μ0: ', m_test)
print('ω0: ', w_test)
print('')

x_start = np.concatenate((m_test, w_test, nm_test, nw_test), axis=None) 
x_max = np.concatenate((np.ones(Bm+Bw), np.ones(Bm+Bw)*np.inf), axis=None)
print('Upper Bound', x_max)
solution = least_squares(SSE, x_start, bounds=(0, x_max), verbose=1) 

μ_star = solution.x[:Bm]
ω_star = solution.x[Bm:Bw]
print('')
print('μ*', μ_star) 
print('ω*', ω_star) 
print('Loss:', round(solution.cost,6))

In [ ]:
# Computing steady state
Nm, Nw, Pbm, Pbw, τm, τw, αm, αw = steady_state(μ_star, ω_star, True)
print('Masses: ', round(Nm), round(Nw))
print('PMF check:', round(sum(Pbm),3), round(sum(Pbw),3))
print('Tightness: ', round(τm,2), round(τw,2))
print('Alphas: ', round(αm,2), round(αw,2)) 

ρm = sum([(1 - Fm.cdf(ω_star[b]))*Pbw[b] for b in range(Bw)])
ρw = sum([(1 - Fw.cdf(μ_star[b]))*Pbm[b] for b in range(Bm)])
print('Average Pr(right-swipe): ', round(ρm, 2), round(ρw, 2))

In [ ]:
# Plotting Marginal PDFs
fig, axs = plt.subplots(2,3,figsize=(18, 8), constrained_layout=True)
x = np.linspace(0, 1, 5000)

axs[0,0].set_xlim(0,1)
axs[0,0].set_ylim(0,1.1*max(Fm.pdf(x)))
axs[0,0].plot(x, Fm.pdf(x), color='tab:blue') 

axs[1,0].set_xlim(0,1)
axs[1,0].set_ylim(0,1.1*max(Fw.pdf(x)))
axs[1,0].plot(x, Fw.pdf(x), color='tab:pink')

# Plotting Marginal PMFs 
axs[0,1].set_xlim(0,Bm+1)
axs[0,1].bar(bm_vals, Pbm, color='tab:blue') 

axs[1,1].set_xlim(0,Bw+1)
axs[1,1].bar(bw_vals, Pbw, color='tab:pink') 

axs[0,1].set(xlabel=r'Budget ($b$)', ylabel=r'Pr($b_m$=b)') 
axs[1,1].set(xlabel=r'Budget ($b$)', ylabel=r'Pr($b_w$=b)') 

#axs[0,0].set_title('Men',loc='center')
axs[0,0].set(xlabel=r'Attractiveness ($\theta$)', ylabel=r'$f(\theta)$')

#axs[0,1].set_title('Women',loc='center')
axs[1,0].set(xlabel=r'Attractiveness ($\theta$)', ylabel=r'$f(\theta)$')


for i in range(2):
    B = Bm if i==0 else Bw
    b_vals = bm_vals if i==0 else bw_vals
    θ_vals = μ_star if i==0 else ω_star
    
    axs[i,2].set_xlim(1,B)
    axs[i,2].set_ylim(0,1)
    axs[i,2].plot(b_vals, θ_vals, color='k')
    axs[i,2].fill_between(b_vals, 0, θ_vals, color='red', alpha=0.05)
    axs[i,2].fill_between(b_vals, θ_vals, 1, color='green', alpha=0.1)

axs[0,2].set(xlabel=r'Budget ($b_m$)', ylabel=r'Attractiveness ($\theta_w$)')
axs[1,2].set(xlabel=r'Budget ($b_w$)', ylabel=r'Attractiveness ($\theta_m$)')
fig.suptitle('Figure 5: Two-Sided Swiping Rules and Steady State ')
plt.savefig('../../figures/mkt-cs.png')
plt.show()